<a href="https://colab.research.google.com/github/LanaUSP/LanaUSP/blob/main/Projeto_Lana_(b3fileparser).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Escopo / Resumo

Projeto de otimização de carteiras de investimentos

Biblioteca utilizada: [b3fileparser](https://github.com/codigoquant/b3fileparser)

[Baixar séries históricas](https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/)

## Instalar e Importar a Biblioteca

Primeiramente, você precisa instalar a biblioteca **b3fileparser**. Você pode fazer isso usando o comando ```pip install b3fileparser```.

In [ ]:
!pip install b3fileparser

## Coletar Dados das Empresas

Usando a biblioteca b3fileparser para obter dados das empresas da bolsa. Aqui está um exemplo de como coletar esses dados:

In [ ]:
import pandas as pd
from b3fileparser.b3parser import B3Parser

# Criar o parser com engine pandas
parser = B3Parser.create_parser(engine='pandas')

# Ler o arquivo de cotações históricas
dados_b3 = parser.read_b3_file('COTAHIST_A2024.TXT')
print(dados_b3.head())


   TIPO_DE_REGISTRO DATA_DO_PREGAO  CODIGO_BDI CODIGO_DE_NEGOCIACAO  \
1                 1     2024-01-02         2.0                AALR3   
2                 1     2024-01-02         2.0                ABCB4   
3                 1     2024-01-02         2.0                ABEV3   
4                 1     2024-01-02         2.0                BBDC3   
5                 1     2024-01-02         2.0                ALPA3   

   TIPO_DE_MERCADO NOME_DA_EMPRESA ESPECIFICACAO_DO_PAPEL  \
1               10          ALLIAR             ON      NM   
2               10      ABC BRASIL             PN  EJS N2   
3               10       AMBEV S/A                 ON  EJ   
4               10        BRADESCO             ON  EJ  N1   
5               10      ALPARGATAS             ON      N1   

   PRAZO_EM_DIAS_DO_MERCADO_A_TERMO MOEDA_DE_REFERENCIA  PRECO_DE_ABERTURA  \
1                               NaN                  R$              10.20   
2                               NaN               

## Processamento dos Dados

Vamos filtrar os dados para manter apenas as informações relevantes das empresas e calcular as métricas financeiras necessárias.

In [ ]:
# Filtrar dados para obter apenas ações de mercado à vista (TIPO_DE_MERCADO == 'VISTA')
dados_b3_vista = dados_b3[dados_b3['TIPO_DE_MERCADO'] == 'VISTA']

# Selecionar colunas relevantes
dados_empresas = dados_b3_vista[['CODIGO_DE_NEGOCIACAO', 'NOME_DA_EMPRESA', 'PRECO_DE_ABERTURA', 'PRECO_MAXIMO', 'PRECO_MINIMO', 'PRECO_MEDIO', 'PRECO_ULTIMO_NEGOCIO', 'NUMERO_DE_NEGOCIOS', 'QUANTIDADE_NEGOCIADA', 'VOLUME_TOTAL_NEGOCIADO']]

# Calcular métricas financeiras
dados_empresas['Lucro'] = dados_empresas['PRECO_MEDIO']  # Exemplo de lucro
dados_empresas['Crescimento'] = dados_empresas['PRECO_ULTIMO_NEGOCIO'] / dados_empresas['PRECO_DE_ABERTURA'] - 1  # Crescimento percentual
dados_empresas['Estabilidade'] = dados_empresas['NUMERO_DE_NEGOCIOS']  # Exemplo de estabilidade

# Remover colunas originais não necessárias
dados_empresas = dados_empresas[['CODIGO_DE_NEGOCIACAO', 'NOME_DA_EMPRESA', 'Lucro', 'Crescimento', 'Estabilidade']]
print(dados_empresas.head())


Empty DataFrame
Columns: [CODIGO_DE_NEGOCIACAO, NOME_DA_EMPRESA, Lucro, Crescimento, Estabilidade]
Index: []


## Funções para AHP Gaussiano

Vamos verificar e corrigir as funções de normalização e cálculo da matriz de comparação.

In [ ]:
import numpy as np
from scipy.stats import norm

# Função para normalizar os dados
def normalize_data(df):
    return (df - df.min()) / (df.max() - df.min())

# Função para calcular a matriz de comparações
def calculate_comparison_matrix(data, weights):
    normalized_data = normalize_data(data)
    matrix = np.zeros((len(data), len(data)))
    for i in range(len(data)):
        for j in range(len(data)):
            matrix[i][j] = sum(weights * norm.cdf(normalized_data.iloc[i] - normalized_data.iloc[j]))
    return matrix

# Função para calcular os pesos das empresas
def calculate_company_weights(matrix):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_eigval_index = np.argmax(eigvals)
    weights = eigvecs[:, max_eigval_index]
    weights = weights / weights.sum()
    return weights.real


## Aplicação do Método

Vamos aplicar as funções corrigidas ao nosso conjunto de dados.

In [ ]:
# Definir os critérios e suas ponderações
criteria = ['Lucro', 'Crescimento', 'Estabilidade']
weights = [0.4, 0.3, 0.3]  # Exemplo de ponderações

# Filtrar e preparar os dados das empresas
data = dados_empresas[criteria].dropna()

# Calcular a matriz de comparações e os pesos das empresas
comparison_matrix = calculate_comparison_matrix(data, weights)

# Verificar se a matriz de comparação foi calculada corretamente
if comparison_matrix.size == 0:
    raise ValueError("A matriz de comparação está vazia. Verifique os dados de entrada.")

company_weights = calculate_company_weights(comparison_matrix)

# Adicionar os pesos ao dataframe
dados_empresas['Peso'] = company_weights

# Exibir o ranking das empresas
dados_empresas = dados_empresas.sort_values(by='Peso', ascending=False)
print(dados_empresas)


ValueError: A matriz de comparação está vazia. Verifique os dados de entrada.

## Otimização com Hill Climbing

Vamos usar os pesos calculados para otimizar a carteira de investimentos usando Hill Climbing.

In [ ]:
import random

# Definir a função de custo
def cost_function(portfolio, returns, risks):
    return portfolio.dot(returns) - portfolio.dot(risks)

# Implementar o algoritmo Hill Climbing
def hill_climbing(returns, risks, num_assets, max_iters=1000):
    current_solution = np.random.rand(num_assets)
    current_solution = current_solution / current_solution.sum()
    current_cost = cost_function(current_solution, returns, risks)

    for i in range(max_iters):
        neighbor = current_solution + np.random.normal(0, 0.1, num_assets)
        neighbor = np.clip(neighbor, 0, 1)
        neighbor = neighbor / neighbor.sum()

        neighbor_cost = cost_function(neighbor, returns, risks)

        if neighbor_cost > current_cost:
            current_solution = neighbor
            current_cost = neighbor_cost

    return current_solution, current_cost

# Exemplo de dados fictícios de retornos e riscos
returns = dados_empresas['Lucro'].values
risks = 1 - dados_empresas['Estabilidade'].values  # Supomos que maior estabilidade significa menor risco

# Executar o algoritmo Hill Climbing
best_portfolio, best_cost = hill_climbing(returns, risks, len(returns))
print("Melhor portfólio:", best_portfolio)
print("Melhor custo:", best_cost)


Melhor portfólio: []
Melhor custo: 0.0


## Ajuste dos Pesos com Redes Neurais

Para ajustar os pesos dos ativos utilizando redes neurais, vamos treinar um modelo de rede neural com os dados disponíveis.

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

# Preparar os dados de treinamento
X_train = dados_empresas[criteria].values
y_train = dados_empresas['Peso'].values.reshape(-1, 1)

# Definir a arquitetura da rede neural
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
model.fit(X_train, y_train, epochs=50, batch_size=10)

# Prever os pesos dos ativos com a rede neural treinada
predictions = model.predict(X_train)

dados_empresas['Predicted_Peso'] = predictions

print("Previsões de pesos para os ativos:", dados_empresas[['CODIGO_DE_NEGOCIACAO', 'Predicted_Peso']])
